<a href="https://colab.research.google.com/github/yohancsx/SmartMask/blob/main/tf_lite/BleSenseSmartMaskML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Notebook for the BLE Smart Mask Project
Trains the tf model to recognize various pressure changes and correlate to behaviors which are talking, coughing, heavy breathing and light breathing.
Also converts the model into arduino-friendly format.

In [ ]:
#set up the notebook
!apt-get -qq install xxd
!pip install pandas numpy matplotlib
!pip install tensorflow==2.0.0-rc1

In [ ]:
#imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
#load all the data into seperate dataframes

#the name for the breathing data
breathingFile = "regular_breathing.csv"

#the name for the coughing data
coughingFile = "coughing.csv"

#the filename for the talking data
talkingFile = "talking.csv"

#the filename for the deep breathing data
deepBreathingFile = "deep_breathing.csv"

#the filename for the combined data
combinedFile = "mixed.csv"

#plot some of the data on the same axes
dfTalk = pd.read_csv("/content/" + talkingFile)
dfBreathe = pd.read_csv("/content/" + breathingFile)
dfCough = pd.read_csv("/content/" + coughingFile)

display(dfTalk)


In [ ]:
#plot some of the sample data

plt.rcParams["figure.figsize"] = (20,10)

plt.plot(index, df['aX'], 'g.', label='x', linestyle='solid', marker=',')
plt.plot(index, df['aY'], 'b.', label='y', linestyle='solid', marker=',')
plt.plot(index, df['aZ'], 'r.', label='z', linestyle='solid', marker=',')
plt.title("coughing Data")
plt.xlabel("Sample #")
plt.ylabel("Acceleration (G)")
plt.legend()
plt.show()

plt.plot(index, df['gX'], 'g.', label='x', linestyle='solid', marker=',')
plt.plot(index, df['gY'], 'b.', label='y', linestyle='solid', marker=',')
plt.plot(index, df['gZ'], 'r.', label='z', linestyle='solid', marker=',')
plt.title("Gyroscope")
plt.xlabel("Sample #")
plt.ylabel("Gyroscope (deg/sec)")
plt.legend()
plt.show()
